In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "X-Linked_Lymphoproliferative_Syndrome"
cohort = "GSE211445"

# Input paths
in_trait_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome"
in_cohort_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome/GSE211445"

# Output paths
out_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE211445.csv"
out_gene_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE211445.csv"
out_clinical_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE211445.csv"
json_path = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE211445-GPL10558_series_matrix.txt.gz', 'GSE211445_family.soft.gz']
Identified SOFT files: ['GSE211445_family.soft.gz']
Identified matrix files: ['GSE211445-GPL10558_series_matrix.txt.gz']

Background Information:
!Series_title	"C99R mutation in IRF4 drives a novel gain of function binding and gene upregulation in classical Hodgkin lymphoma"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"

Sample Characteristics Dictionary:
{0: ['cell line: BJAB'], 1: ['group: Control 0hrs', 'group: Control 6hrs', 'group: Control 24hrs', 'group: Control 48hrs', 'group: IRF4WT 0hrs', 'group: IRF4WT 6hrs', 'group: IRF4WT 24hrs', 'group: IRF4WT 48hrs', 'group: IRF4C99R 0hrs', 'group: IRF4C99R 6hrs', 'group: IRF4C99R 24hrs', 'group: IRF4C99R 48hrs']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information and available files, this dataset likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Based on the sample characteristics dictionary:

# For trait: The group information in row 1 contains treatment information (IRF4WT vs IRF4C99R vs Control)
# which can be used to derive the trait (X-linked lymphoproliferative syndrome)
trait_row = 1

# Age data is not available in the sample characteristics
age_row = None

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert the group information to binary trait data.
    IRF4C99R mutation is associated with X-Linked Lymphoproliferative Syndrome, 
    so samples with IRF4C99R will be coded as 1, others as 0.
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert based on the presence of IRF4C99R
    if "IRF4C99R" in value:
        return 1  # Disease/mutation
    elif "Control" in value or "IRF4WT" in value:
        return 0  # Control or wild type
    else:
        return None  # Unknown or not applicable

# Since age and gender are not available, creating placeholder functions
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Determine trait availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # Using the clinical_data from previous steps
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_df)
    print("Clinical data preview:", preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview: {'GSM6505368': [0.0], 'GSM6505369': [0.0], 'GSM6505370': [0.0], 'GSM6505371': [0.0], 'GSM6505372': [0.0], 'GSM6505373': [0.0], 'GSM6505374': [0.0], 'GSM6505375': [0.0], 'GSM6505376': [1.0], 'GSM6505377': [1.0], 'GSM6505378': [1.0], 'GSM6505379': [1.0], 'GSM6505380': [0.0], 'GSM6505381': [0.0], 'GSM6505382': [0.0], 'GSM6505383': [0.0], 'GSM6505384': [0.0], 'GSM6505385': [0.0], 'GSM6505386': [0.0], 'GSM6505387': [0.0], 'GSM6505388': [1.0], 'GSM6505389': [1.0], 'GSM6505390': [1.0], 'GSM6505391': [1.0]}
Clinical data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE211445.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene expression data shape: (47323, 24)


### Step 4: Gene Identifier Review

In [5]:
# The identifiers beginning with "ILMN_" indicate these are Illumina BeadArray probe IDs,
# not standard human gene symbols. These probe IDs need to be mapped to their corresponding
# gene symbols for biological interpretation and analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
              GSM6505368  GSM6505369  GSM6505370  GSM6505371  GSM6505372
ID                                                                      
ILMN_1343291  60335.4400  56748.2800  54508.7400  53976.1000  65779.4800
ILMN_1343295  28342.9800  27970.3300  27209.1000  20349.3500  32162.1300
ILMN_1651199    140.8024    120.8438    121.3026     96.2961    133.4001
ILMN_1651209    185.5267    167.3002    169.0416    167.9951    186.3865
ILMN_1651210    154.0286    129.2618    137.3497    128.1371    136.0509



Platform information:
!Series_title = C99R mutation in IRF4 drives a novel gain of function binding and gene upregulation in classical Hodgkin lymphoma
!Platform_title = Illumina HumanHT-12 V4.0 expression beadchip
!Platform_description = The HumanHT-12 v4 Expression BeadChip provides high throughput processing of 12 samples per BeadChip without the need for expensive, specialized automation. The BeadChip is designed to support flexible usage across a wide-spectrum of experiments.
!Platform_description =
!Platform_description = The updated content on the HumanHT-12 v4 Expression BeadChips provides more biologically meaningful results through genome-wide transcriptional coverage of well-characterized genes, gene candidates, and splice variants.
!Platform_description =
!Platform_description = Each array on the HumanHT-12 v4 Expression BeadChip targets more than 31,000 annotated genes with more than 47,000 probes derived from the National Center for Biotechnology Information Reference Se


Gene annotation columns:
['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']

Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Ac

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene_annotation contain probe IDs and gene symbols
prob_col = 'ID'  # The probe identifiers column
gene_col = 'Symbol'  # The gene symbols column

# 2. Get the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Check the first few rows of mapping_df to verify it has what we need
print("Gene mapping dataframe (first 5 rows):")
print(mapping_df.head())
print(f"Mapping dataframe shape: {mapping_df.shape}")

# 3. Convert probe-level measurements to gene-level expression data
# This applies the gene mapping and handles many-to-many relationships
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Verify the gene-level expression data
print("\nGene-level expression dataframe (first 5 rows, first 3 columns):")
print(gene_data.iloc[:5, :3])
print(f"Gene expression data shape after mapping: {gene_data.shape}")

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to: {out_gene_data_file}")


Gene mapping dataframe (first 5 rows):
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Mapping dataframe shape: (44837, 2)

Gene-level expression dataframe (first 5 rows, first 3 columns):
       GSM6505368  GSM6505369  GSM6505370
Gene                                     
A1BG    358.11860    313.0659   286.89630
A1CF    405.94150    395.7438   371.76720
A26C3   404.48630    434.8431   426.75660
A2BP1   461.89607    480.3532   434.01131
A2LD1   259.27540    206.9460   203.19940
Gene expression data shape after mapping: (21464, 24)



Gene expression data saved to: ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE211445.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (20259, 24)
First few gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Normalized gene data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE211445.csv
Loaded clinical data:
                                       GSM6505368  GSM6505369  GSM6505370  \
X-Linked_Lymphoproliferative_Syndrome         0.0         0.0         0.0   

                                       GSM6505371  GSM6505372  GSM6505373  \
X-Linked_Lymphoproliferative_Syndrome         0.0         0.0         0.0   

                                       GSM6505374  GSM6505375  GSM6505376  \
X-Linked_Lymphoproliferative_Syndrome         0.0         0.0         1.0   

                                       GSM6505377  ...  GSM6505382  \
X-Linked_Lymphoproliferative_Syndrome         1.0  ...         0.0   

                                       GSM6505383  GSM6505384  GSM6505385  \
X-Linked_Lymphoproliferative_Syndrome         0.0         0.0         0.0   

                                       GSM6505386  GSM6505387  GSM6505388  \
X-Linked_Lymphoprolifera

Linked data shape after handling missing values: (24, 20260)
For the feature 'X-Linked_Lymphoproliferative_Syndrome', the least common label is '1.0' with 8 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'X-Linked_Lymphoproliferative_Syndrome' in this dataset is fine.

Is trait biased: False
Data quality check result: Usable


Linked data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE211445.csv
